In [1]:
from modules.spec_dataset import *
from modules.train_prep import *

In [2]:
class SpectroMLP(nn.Module):
    def __init__(self, input_size, num_classes=4):
        super(SpectroMLP, self).__init__()
        # Two hidden layers with a moderate number of neurons
        self.fc1 = nn.Linear(input_size, 512)  # First hidden layer
        self.fc2 = nn.Linear(512, 256)         # Second hidden layer
        self.fc3 = nn.Linear(256, num_classes) # Output layer

    def forward(self, x):
        # Flatten the input tensor if not already flattened
        x = x.view(x.size(0), -1)
        # Forward pass through the network
        x = F.relu(self.fc1(x))  # Activation function for the first hidden layer
        x = F.relu(self.fc2(x))  # Activation function for the second hidden layer
        x = self.fc3(x)          # No activation function for output layer
        return x

In [3]:
if torch.cuda.is_available():
    torch.cuda.set_device(cuda_device)
elif torch.backends.mps.is_available():
    mps_device = torch.device("mps")

data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

# Setup datasets and loaders
dsets = {
    'train': SpectroDataset(train_dir, train_behav_file, data_transforms['train']),
    'val': SpectroDataset(val_dir, val_behav_file, data_transforms['val'])
}
dset_sizes = {split: len(dsets[split]) for split in ['train', 'val']}

dset_loaders = {}
for split in ['train', 'val']:
    targets = np.array([dsets[split].get_label(i) for i in range(len(dsets[split]))])
    class_counts = dsets[split].get_class_counts()
    class_weights = np.array([1.0 / class_counts[label] if class_counts[label] > 0 else 0 for label in targets])
    sampler = WeightedRandomSampler(class_weights, num_samples=len(class_weights), replacement=True)
    dset_loaders[split] = torch.utils.data.DataLoader(dsets[split], batch_size=b_size, num_workers=0, sampler=sampler)
    print('done making loader:', split)

# Initialize model, criterion, and optimizer
model_ft = SpectroMLP(input_size=108360, num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.01, weight_decay=1e-5)

# Device configuration with MPS support
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model_ft.to(device)
criterion.to(device)
print(f"Training on {device}")

# Train model
model_ft, accuracies, losses = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dset_loaders, dset_sizes, num_epochs=n_epochs)

# Output results
for split in ['train', 'val']:
    print(split, 'accuracies by epoch:', accuracies[split])
    print(split, 'losses by epoch:', losses[split])


100%|██████████| 192/192 [00:06<00:00, 27.85it/s]


Class counts:  {'HEHF': 2482, 'HELF': 238, 'LEHF': 1366, 'LELF': 874}


100%|██████████| 48/48 [00:01<00:00, 27.36it/s]


Class counts:  {'HEHF': 507, 'HELF': 100, 'LEHF': 533, 'LELF': 100}
done making loader: train
done making loader: val
Training on mps
----------
Epoch 0/9
----------
LR is set to 0.0001
Reached batch iteration 0
Reached batch iteration 10
Reached batch iteration 20
Reached batch iteration 30
Reached batch iteration 40
Reached batch iteration 50
Reached batch iteration 60
Reached batch iteration 70
Reached batch iteration 80
